<a href="https://colab.research.google.com/github/AaronScherf/gxe-gee-lmm/blob/master/6_LMM_replication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Grain Yields with a Pedigree-Based Reaction Norm GxE Model using the BGLR Package in R

From A Pedigree-Based Reaction Norm Model for Prediction of Cotton Yield in Multienvironment Trials

https://dl.sciencesocieties.org/publications/cs/abstracts/55/3/1143?access=0&view=pdf

## Install Necessary Packages and Mount Drive

In [0]:
# To host on a local runtime:
# Run the following from the command line or miniconda shell on your local machine:

# pip install jupyter_http_over_ws
# jupyter serverextension enable --py jupyter_http_over_ws

# conda create -n gxe_gee python=3.7 ipykernel jupyter anaconda
# conda activate gxe_gee
# ipython kernel install --name gxe_gee --user

# jupyter notebook \ --NotebookApp.allow_origin='https://colab.research.google.com' \ --port=8888 \ --NotebookApp.port_retries=0
# jupyter notebook  --NotebookApp.allow_origin='https://colab.research.google.com'  --port=8888  --NotebookApp.port_retries=0


In [0]:
# Installing rpy2 package through Python to run R code

#!pip install rpy2

# If pip install doesn't work, try:

# !conda install -c r rpy2

# If the conda install line doesn't work, or if you have issues on your locally hosted runtime, try the following:
# https://anaconda.zendesk.com/hc/en-us/articles/360023857134-Setting-up-rpy2-on-Windows


## Mount Drive or Set Data Path

In [77]:
# Mount the Colab notebook in Google Drive
# Only if running from a hosted runtime in Colab, ignore if running locally

from google.colab import drive # import drive from google colab

drive.mount("/content/drive")  # mount the google drive at /content/drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [78]:
# Search for the root directory of the project in Drive and save as a path string

parent_dir_name = 'GxE with GEE'

for dirpath, subdirs, files in os.walk('/content'):
  if parent_dir_name in subdirs:
    parent_path = dirpath + "/" + parent_dir_name

parent_path

'/content/drive/My Drive/Research/GxE with GEE'

In [79]:
# Set Data Path
## Change the destination to your Drive directory containing the folder 'raw_data.zip'
data_path_end = '/Colab Workspace/Data'
data_path = parent_path + data_path_end
os.chdir(data_path)
os.getcwd()

'/content/drive/My Drive/Research/GxE with GEE/Colab Workspace/Data'

In [0]:
# Locally hosted version, just set the destination manually

# data_path = 'C:/Users/theaa/Downloads'
# os.chdir('C:/Users/theaa/Downloads')
# os.getcwd()

## Initial R Benchmark

In [0]:
#@title Run Benchmark Test { vertical-output: true, display-mode: "form" }
%%R
# R Benchmark 2.5 (06/2008) [Simon Urbanek]
# version 2.5: scaled to get roughly 1s per test, R 2.7.0 @ 2.6GHz Mac Pro
# R Benchmark 2.4 (06/2008) [Simon Urbanek]
# version 2.4 adapted to more recent Matrix package
# R Benchmark 2.3 (21 April 2004)
# Warning: changes are not carefully checked yet!
# version 2.3 adapted to R 1.9.0
# Many thanks to Douglas Bates (bates@stat.wisc.edu) for improvements!
# version 2.2 adapted to R 1.8.0
# version 2.1 adapted to R 1.7.0
# version 2, scaled to get 1 +/- 0.1 sec with R 1.6.2
# using the standard ATLAS library (Rblas.dll)
# on a Pentium IV 1.6 Ghz with 1 Gb Ram on Win XP pro

# revised and optimized for R v. 1.5.x, 8 June 2002
# Requires additionnal libraries: Matrix, SuppDists
# Author : Philippe Grosjean
# eMail  : phgrosjean@sciviews.org
# Web    : http://www.sciviews.org
# License: GPL 2 or above at your convenience (see: http://www.gnu.org)
#
# Several tests are adapted from the Splus Benchmark Test V. 2
# by Stephan Steinhaus (stst@informatik.uni-frankfurt.de) 
# Reference for Escoufier's equivalents vectors (test III.5):
# Escoufier Y., 1970. Echantillonnage dans une population de variables
# aleatoires réelles. Publ. Inst. Statis. Univ. Paris 19 Fasc 4, 1-47.
#
# type source("c:/<dir>/R2.R") to start the test

runs <- 3			# Number of times the tests are executed
times <- rep(0, 15); dim(times) <- c(5,3)
require(Matrix)		# Optimized matrix operations
require(SuppDists)	# Optimized random number generators
#Runif <- rMWC1019	# The fast uniform number generator
Runif <- runif
# If you don't have SuppDists, you can use: Runif <- runif
#a <- rMWC1019(10, new.start=TRUE, seed=492166)	# Init. the generator
#Rnorm <- rziggurat	# The fast normal number generator
# If you don't have SuppDists, you can use: Rnorm <- rnorm
#b <- rziggurat(10, new.start=TRUE)	# Init. the generator
Rnorm <- rnorm
remove("a", "b")
options(object.size=100000000)

cat("\n\n   R Benchmark 2.5\n")
cat("   ===============\n")
cat(c("Number of times each test is run__________________________: ", runs))
cat("\n\n")


cat("   I. Matrix calculation\n")
cat("   ---------------------\n")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (1)
cumulate <- 0; a <- 0; b <- 0
for (i in 1:runs) {
  invisible(gc())
  timing <- system.time({
    a <- matrix(Rnorm(2500*2500)/10, ncol=2500, nrow=2500);
    b <- t(a);
    dim(b) <- c(1250, 5000);
    a <- t(b)
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[1, 1] <- timing
cat(c("Creation, transp., deformation of a 2500x2500 matrix (sec): ", timing, "\n"))
remove("a", "b")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (2)
cumulate <- 0; b <- 0
for (i in 1:runs) {
  a <- abs(matrix(Rnorm(2500*2500)/2, ncol=2500, nrow=2500));
  invisible(gc())
  timing <- system.time({ 
    b <- a^1000 
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[2, 1] <- timing
cat(c("2400x2400 normal distributed random matrix ^1000____ (sec): ", timing, "\n"))
remove("a", "b")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (3)
cumulate <- 0; b <- 0
for (i in 1:runs) {
  a <- Rnorm(7000000)
  invisible(gc())
  timing <- system.time({
    b <- sort(a, method="quick")	# Sort is modified in v. 1.5.x
    # And there is now a quick method that better competes with other packages!!!
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[3, 1] <- timing
cat(c("Sorting of 7,000,000 random values__________________ (sec): ", timing, "\n"))
remove("a", "b")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (4)
cumulate <- 0; b <- 0
for (i in 1:runs) {
  a <- Rnorm(2800*2800); dim(a) <- c(2800, 2800)
  invisible(gc())
  timing <- system.time({
    b <- crossprod(a)		# equivalent to: b <- t(a) %*% a
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[4, 1] <- timing
cat(c("2800x2800 cross-product matrix (b = a' * a)_________ (sec): ", timing, "\n"))
remove("a", "b")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (5)
cumulate <- 0; c <- 0; qra <-0
for (i in 1:runs) {
  a <- new("dgeMatrix", x = Rnorm(2000*2000), Dim = as.integer(c(2000,2000)))
  b <- as.double(1:2000)
  invisible(gc())
  timing <- system.time({
    c <- solve(crossprod(a), crossprod(a,b))
  })[3]
  cumulate <- cumulate + timing
  
  # This is the old method
  #a <- Rnorm(600*600); dim(a) <- c(600,600)
  #b <- 1:600
  #invisible(gc())
  #timing <- system.time({
  #  qra <- qr(a, tol = 1e-7);
  #  c <- qr.coef(qra, b)
  #  #Rem: a little faster than c <- lsfit(a, b, inter=F)$coefficients
  #})[3]
  #cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[5, 1] <- timing
cat(c("Linear regr. over a 3000x3000 matrix (c = a \\ b')___ (sec): ", timing, "\n"))
remove("a", "b", "c", "qra")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

times[ , 1] <- sort(times[ , 1])
cat("                      --------------------------------------------\n")
cat(c("                 Trimmed geom. mean (2 extremes eliminated): ", exp(mean(log(times[2:4, 1]))), "\n\n"))

cat("   II. Matrix functions\n")
cat("   --------------------\n")
if (R.Version()$os == "Win32") flush.console()

# (1)
cumulate <- 0; b <- 0
for (i in 1:runs) {
  a <- Rnorm(2400000)
  invisible(gc())
  timing <- system.time({
    b <- fft(a)
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[1, 2] <- timing
cat(c("FFT over 2,400,000 random values____________________ (sec): ", timing, "\n"))
remove("a", "b")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (2)
cumulate <- 0; b <- 0
for (i in 1:runs) {
  a <- array(Rnorm(600*600), dim = c(600, 600))
  # Only needed if using eigen.Matrix(): Matrix.class(a)
  invisible(gc())
  timing <- system.time({
  	b <- eigen(a, symmetric=FALSE, only.values=TRUE)$Value
  	# Rem: on my machine, it is faster than:
  #	 b <- La.eigen(a, symmetric=F, only.values=T, method="dsyevr")$Value
  #	 b <- La.eigen(a, symmetric=F, only.values=T, method="dsyev")$Value
  #  b <- eigen.Matrix(a, vectors = F)$Value
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[2, 2] <- timing
cat(c("Eigenvalues of a 640x640 random matrix______________ (sec): ", timing, "\n"))
remove("a", "b")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (3)
cumulate <- 0; b <- 0
for (i in 1:runs) {
  a <- Rnorm(2500*2500); dim(a) <- c(2500, 2500)
  #Matrix.class(a)
  invisible(gc())
  timing <- system.time({
    #b <- determinant(a, logarithm=F)
    # Rem: the following is slower on my computer!
    # b <- det.default(a)
    b <- det(a)
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[3, 2] <- timing
cat(c("Determinant of a 2500x2500 random matrix____________ (sec): ", timing, "\n"))
remove("a", "b")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (4)
cumulate <- 0; b <- 0
for (i in 1:runs) {
  a <- crossprod(new("dgeMatrix", x = Rnorm(3000*3000),
                       Dim = as.integer(c(3000, 3000))))
  invisible(gc())
  #a <- Rnorm(900*900); dim(a) <- c(900, 900)
  #a <- crossprod(a, a)
  timing <- system.time({
    b <- chol(a)
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[4, 2] <- timing
cat(c("Cholesky decomposition of a 3000x3000 matrix________ (sec): ", timing, "\n"))
remove("a", "b")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (5)
cumulate <- 0; b <- 0
for (i in 1:runs) {
  a <- new("dgeMatrix", x = Rnorm(1600*1600), Dim = as.integer(c(1600, 1600)))
  invisible(gc())
  #a <- Rnorm(400*400); dim(a) <- c(400, 400)
  timing <- system.time({
  #  b <- qr.solve(a)
    # Rem: a little faster than
    b <- solve(a)
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[5, 2] <- timing
cat(c("Inverse of a 1600x1600 random matrix________________ (sec): ", timing, "\n"))
remove("a", "b")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

times[ , 2] <- sort(times[ , 2])
cat("                      --------------------------------------------\n")
cat(c("                Trimmed geom. mean (2 extremes eliminated): ", exp(mean(log(times[2:4, 2]))), "\n\n"))

cat("   III. Programmation\n")
cat("   ------------------\n")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (1)
cumulate <- 0; a <- 0; b <- 0; phi <- 1.6180339887498949
for (i in 1:runs) {
  a <- floor(Runif(3500000)*1000)
  invisible(gc())
  timing <- system.time({
    b <- (phi^a - (-phi)^(-a))/sqrt(5)
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[1, 3] <- timing
cat(c("3,500,000 Fibonacci numbers calculation (vector calc)(sec): ", timing, "\n"))
remove("a", "b", "phi")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (2)
cumulate <- 0; a <- 3000; b <- 0
for (i in 1:runs) {
  invisible(gc())
  timing <- system.time({
    b <- rep(1:a, a); dim(b) <- c(a, a);
    b <- 1 / (t(b) + 0:(a-1))
    # Rem: this is twice as fast as the following code proposed by R programmers
    # a <- 1:a; b <- 1 / outer(a - 1, a, "+")
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[2, 3] <- timing
cat(c("Creation of a 3000x3000 Hilbert matrix (matrix calc) (sec): ", timing, "\n"))
remove("a", "b")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (3)
cumulate <- 0; c <- 0
gcd2 <- function(x, y) {if (sum(y > 1.0E-4) == 0) x else {y[y == 0] <- x[y == 0]; Recall(y, x %% y)}}
for (i in 1:runs) {
  a <- ceiling(Runif(400000)*1000)
  b <- ceiling(Runif(400000)*1000)
  invisible(gc())
  timing <- system.time({	  
    c <- gcd2(a, b)                            # gcd2 is a recursive function
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[3, 3] <- timing
cat(c("Grand common divisors of 400,000 pairs (recursion)__ (sec): ", timing, "\n"))
remove("a", "b", "c", "gcd2")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (4)
cumulate <- 0; b <- 0
for (i in 1:runs) {
  b <- rep(0, 500*500); dim(b) <- c(500, 500)
  invisible(gc())
  timing <- system.time({
  	# Rem: there are faster ways to do this
  	# but here we want to time loops (220*220 'for' loops)! 
    for (j in 1:500) {
      for (k in 1:500) {
        b[k,j] <- abs(j - k) + 1
      }
    }
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[4, 3] <- timing
cat(c("Creation of a 500x500 Toeplitz matrix (loops)_______ (sec): ", timing, "\n"))
remove("b", "j", "k")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (5)
cumulate <- 0; p <- 0; vt <- 0; vr <- 0; vrt <- 0; rvt <- 0; RV <- 0; j <- 0; k <- 0;
x2 <- 0; R <- 0; Rxx <- 0; Ryy <- 0; Rxy <- 0; Ryx <- 0; Rvmax <- 0
# Calculate the trace of a matrix (sum of its diagonal elements)
Trace <- function(y) {sum(c(y)[1 + 0:(min(dim(y)) - 1) * (dim(y)[1] + 1)], na.rm=FALSE)}
for (i in 1:runs) {
  x <- abs(Rnorm(45*45)); dim(x) <- c(45, 45)
  invisible(gc())
  timing <- system.time({
    # Calculation of Escoufier's equivalent vectors
    p <- ncol(x)
    vt <- 1:p                                  # Variables to test
    vr <- NULL                                 # Result: ordered variables
    RV <- 1:p                                  # Result: correlations
    vrt <- NULL
    for (j in 1:p) {                           # loop on the variable number
      Rvmax <- 0
      for (k in 1:(p-j+1)) {                   # loop on the variables
        x2 <- cbind(x, x[,vr], x[,vt[k]])
        R <- cor(x2)                           # Correlations table
        Ryy <- R[1:p, 1:p]
        Rxx <- R[(p+1):(p+j), (p+1):(p+j)]
        Rxy <- R[(p+1):(p+j), 1:p]
        Ryx <- t(Rxy)
        rvt <- Trace(Ryx %*% Rxy) / sqrt(Trace(Ryy %*% Ryy) * Trace(Rxx %*% Rxx)) # RV calculation
        if (rvt > Rvmax) {
          Rvmax <- rvt                         # test of RV
          vrt <- vt[k]                         # temporary held variable
        }
      }
      vr[j] <- vrt                             # Result: variable
      RV[j] <- Rvmax                           # Result: correlation
      vt <- vt[vt!=vr[j]]                      # reidentify variables to test
    }
  })[3]
  cumulate <- cumulate + timing
}
times[5, 3] <- timing
cat(c("Escoufier's method on a 45x45 matrix (mixed)________ (sec): ", timing, "\n"))
remove("x", "p", "vt", "vr", "vrt", "rvt", "RV", "j", "k")
remove("x2", "R", "Rxx", "Ryy", "Rxy", "Ryx", "Rvmax", "Trace") 
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

times[ , 3] <- sort(times[ , 3])
cat("                      --------------------------------------------\n")
cat(c("                Trimmed geom. mean (2 extremes eliminated): ", exp(mean(log(times[2:4, 3]))), "\n\n\n"))

cat(c("Total time for all 15 tests_________________________ (sec): ", sum(times), "\n"))
cat(c("Overall mean (sum of I, II and III trimmed means/3)_ (sec): ", exp(mean(log(times[2:4, ]))), "\n"))
remove("cumulate", "timing", "times", "runs", "i")
cat("                      --- End of test ---\n\n")   



   R Benchmark 2.5
Number of times each test is run__________________________:  3

   I. Matrix calculation
   ---------------------
Creation, transp., deformation of a 2500x2500 matrix (sec):  0.865000000000085 
2400x2400 normal distributed random matrix ^1000____ (sec):  0.636666666666618 
Sorting of 7,000,000 random values__________________ (sec):  1.00933333333342 
2800x2800 cross-product matrix (b = a' * a)_________ (sec):  0.940333333333153 
Linear regr. over a 3000x3000 matrix (c = a \ b')___ (sec):  0.717999999999999 
                      --------------------------------------------
                 Trimmed geom. mean (2 extremes eliminated):  0.835873957160623 

   II. Matrix functions
   --------------------
FFT over 2,400,000 random values____________________ (sec):  0.392666666666779 
Eigenvalues of a 640x640 random matrix______________ (sec):  0.664333333333161 
Determinant of a 2500x2500 random matrix____________ (sec):  0.429999999999836 
Cholesky decomposition of a 3

## Install RPy2 and Activate

In [82]:
# Install packages
!pip install tzlocal
!pip install simplegeneric

In [0]:
# import necessary libraries in Python
import pandas as pd
import numpy as np
import os

In [84]:
# activate R magic
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [85]:
import rpy2
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

# import R's "base" package
base = importr('base')

# import rpy2's package module
import rpy2.robjects.packages as rpackages

# import R's utility package
utils = rpackages.importr('utils')

# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/vectors.py:927: UserWarning: R object inheriting from "POSIXct" but without attribute "tzone".
  warnings.warn('R object inheriting from "POSIXct" but without '


<rpy2.rinterface.NULLType object at 0x7f63d38aa408> [RTYPES.NILSXP]

In [0]:
# Import the pandas2ri functions and activate
from rpy2.robjects import pandas2ri
pandas2ri.activate()

In [0]:
%%capture
%%R # Install other R Packages
# May result in RunTimeWarnings, these can be ignored
install.packages('dplyr')
install.packages('tidyverse')
install.packages('caret')
install.packages("VIM")
install.packages('BGLR')

In [0]:
%%R 
require("dplyr")
require('BGLR')
require('tidyverse')
require('caret')
require('VIM')

In [0]:
%%R -i data_path
# Import the parent_path object from Python and use to set the R working directory
setwd(data_path)
getwd()

## Change BLAS to OpenBLAS

In [0]:
# Switch to OpenBLAS for faster linear algebra computation
# https://www.r-bloggers.com/for-faster-r-use-openblas-instead-better-than-atlas-trivial-to-switch-to-on-ubuntu/
# http://brettklamer.com/diversions/statistical/faster-blas-in-r/
# https://stackoverflow.com/questions/12249089/how-to-use-numpy-with-openblas-instead-of-atlas-in-ubuntu

In [0]:
# OpenBLAS Source
# GitHub: https://github.com/xianyi/OpenBLAS
# http://www.openblas.net/

In [2]:
!apt-cache search libblas

libblas-dev - Basic Linear Algebra Subroutines 3, static library
libblas3 - Basic Linear Algebra Reference implementations, shared library
libatlas-base-dev - Automatically Tuned Linear Algebra Software, generic static
libatlas3-base - Automatically Tuned Linear Algebra Software, generic shared
libblas-test - Basic Linear Algebra Subroutines 3, testing programs
libblasr - tools for aligning PacBio reads to target sequences
libblasr-dev - tools for aligning PacBio reads to target sequences (development files)
libopenblas-base - Optimized BLAS (linear algebra) library (shared library)
libopenblas-dev - Optimized BLAS (linear algebra) library (development files)


In [3]:
# install OpenBLAS
!sudo apt-get install libopenblas-base
# install ATLAS
!sudo apt-get install libatlas3-base liblapack3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopenblas-base is already the newest version (0.2.20+ds-4).
libopenblas-base set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
liblapack3 is already the newest version (3.7.1-4ubuntu1).
liblapack3 set to manually installed.
libatlas3-base is already the newest version (3.10.3-5).
libatlas3-base set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [5]:
# List libblas file if available (note: Colab gives error of no alternatives)
!sudo update-alternatives --list libblas.so.3

update-alternatives: error: no alternatives for libblas.so.3


In [4]:
# Configure libblas and set to openblas/libblas.so.3 (Colab also gives a no alternatives error)
!sudo update-alternatives --config libblas.so.3

update-alternatives: error: no alternatives for libblas.so.3


In [6]:
# If nothing else works, update all library files and select the openblas options for libblas.so and libblas.so.3
!sudo update-alternatives --all

There is only one alternative in link group awk (providing /usr/bin/awk): /usr/bin/mawk
Nothing to configure.
There are 2 choices for the alternative c++ (providing /usr/bin/c++).

  Selection    Path              Priority   Status
------------------------------------------------------------
* 0            /usr/bin/g++       20        auto mode
  1            /usr/bin/clang++   10        manual mode
  2            /usr/bin/g++       20        manual mode

Press <enter> to keep the current choice[*], or type selection number: 
There are 2 choices for the alternative c89 (providing /usr/bin/c89).

  Selection    Path              Priority   Status
------------------------------------------------------------
* 0            /usr/bin/c89-gcc   20        auto mode
  1            /usr/bin/c89-gcc   20        manual mode
  2            /usr/bin/clang     10        manual mode

Press <enter> to keep the current choice[*], or type selection number: 
There are 2 choices for the alternative c99 (p

## Check BLAS Installation

In [16]:
%%R
# http://brettklamer.com/diversions/statistical/faster-blas-in-r/
La_library()

[1] "/usr/lib/x86_64-linux-gnu/openblas/liblapack.so.3"


In [17]:
%%R
extSoftVersion()["BLAS"]

[1] "/usr/lib/x86_64-linux-gnu/openblas/libblas.so.3"


In [26]:
%%R
sessionInfo()

R version 3.6.3 (2020-02-29)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 18.04.3 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/openblas/liblapack.so.3

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] grid      tools     stats     graphics  grDevices utils     datasets 
[8] methods   base     

other attached packages:
 [1] VIM_5.1.1         data.table_1.12.8 colorspace_1.4-1  caret_6.0-86     
 [5] lattice_0.20-41   forcats_0.5.0     stringr_1.4.0     purrr_0.3.3      
 [9] readr_1.3.1       tidyr_1.0.2       tibble_3.0.0      ggplot2_3.3.0    
[13] tidyverse_

In [0]:
# https://www.r-bloggers.com/for-faster-r-use-openblas-instead-better-than-atlas-trivial-to-switch-to-on-ubuntu/

In [18]:
!ps aux | grep exec/R

root       10760  0.0  0.0  39192  6660 ?        S    00:46   0:00 /bin/bash -c ps aux | grep exec/R
root       10762  0.0  0.0  38568  4952 ?        S    00:46   0:00 grep exec/R


In [0]:
!lsof -p 10760 | grep 'blas\|lapack'

### Benchmark R with OpenBLAS

https://mac.r-project.org/benchmarks/

In [35]:
#@title Run Benchmark Test { vertical-output: true, display-mode: "form" }
%%R
# R Benchmark 2.5 (06/2008) [Simon Urbanek]
# version 2.5: scaled to get roughly 1s per test, R 2.7.0 @ 2.6GHz Mac Pro
# R Benchmark 2.4 (06/2008) [Simon Urbanek]
# version 2.4 adapted to more recent Matrix package
# R Benchmark 2.3 (21 April 2004)
# Warning: changes are not carefully checked yet!
# version 2.3 adapted to R 1.9.0
# Many thanks to Douglas Bates (bates@stat.wisc.edu) for improvements!
# version 2.2 adapted to R 1.8.0
# version 2.1 adapted to R 1.7.0
# version 2, scaled to get 1 +/- 0.1 sec with R 1.6.2
# using the standard ATLAS library (Rblas.dll)
# on a Pentium IV 1.6 Ghz with 1 Gb Ram on Win XP pro

# revised and optimized for R v. 1.5.x, 8 June 2002
# Requires additionnal libraries: Matrix, SuppDists
# Author : Philippe Grosjean
# eMail  : phgrosjean@sciviews.org
# Web    : http://www.sciviews.org
# License: GPL 2 or above at your convenience (see: http://www.gnu.org)
#
# Several tests are adapted from the Splus Benchmark Test V. 2
# by Stephan Steinhaus (stst@informatik.uni-frankfurt.de) 
# Reference for Escoufier's equivalents vectors (test III.5):
# Escoufier Y., 1970. Echantillonnage dans une population de variables
# aleatoires réelles. Publ. Inst. Statis. Univ. Paris 19 Fasc 4, 1-47.
#
# type source("c:/<dir>/R2.R") to start the test

runs <- 3			# Number of times the tests are executed
times <- rep(0, 15); dim(times) <- c(5,3)
require(Matrix)		# Optimized matrix operations
require(SuppDists)	# Optimized random number generators
#Runif <- rMWC1019	# The fast uniform number generator
Runif <- runif
# If you don't have SuppDists, you can use: Runif <- runif
#a <- rMWC1019(10, new.start=TRUE, seed=492166)	# Init. the generator
#Rnorm <- rziggurat	# The fast normal number generator
# If you don't have SuppDists, you can use: Rnorm <- rnorm
#b <- rziggurat(10, new.start=TRUE)	# Init. the generator
Rnorm <- rnorm
remove("a", "b")
options(object.size=100000000)

cat("\n\n   R Benchmark 2.5\n")
cat("   ===============\n")
cat(c("Number of times each test is run__________________________: ", runs))
cat("\n\n")


cat("   I. Matrix calculation\n")
cat("   ---------------------\n")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (1)
cumulate <- 0; a <- 0; b <- 0
for (i in 1:runs) {
  invisible(gc())
  timing <- system.time({
    a <- matrix(Rnorm(2500*2500)/10, ncol=2500, nrow=2500);
    b <- t(a);
    dim(b) <- c(1250, 5000);
    a <- t(b)
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[1, 1] <- timing
cat(c("Creation, transp., deformation of a 2500x2500 matrix (sec): ", timing, "\n"))
remove("a", "b")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (2)
cumulate <- 0; b <- 0
for (i in 1:runs) {
  a <- abs(matrix(Rnorm(2500*2500)/2, ncol=2500, nrow=2500));
  invisible(gc())
  timing <- system.time({ 
    b <- a^1000 
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[2, 1] <- timing
cat(c("2400x2400 normal distributed random matrix ^1000____ (sec): ", timing, "\n"))
remove("a", "b")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (3)
cumulate <- 0; b <- 0
for (i in 1:runs) {
  a <- Rnorm(7000000)
  invisible(gc())
  timing <- system.time({
    b <- sort(a, method="quick")	# Sort is modified in v. 1.5.x
    # And there is now a quick method that better competes with other packages!!!
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[3, 1] <- timing
cat(c("Sorting of 7,000,000 random values__________________ (sec): ", timing, "\n"))
remove("a", "b")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (4)
cumulate <- 0; b <- 0
for (i in 1:runs) {
  a <- Rnorm(2800*2800); dim(a) <- c(2800, 2800)
  invisible(gc())
  timing <- system.time({
    b <- crossprod(a)		# equivalent to: b <- t(a) %*% a
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[4, 1] <- timing
cat(c("2800x2800 cross-product matrix (b = a' * a)_________ (sec): ", timing, "\n"))
remove("a", "b")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (5)
cumulate <- 0; c <- 0; qra <-0
for (i in 1:runs) {
  a <- new("dgeMatrix", x = Rnorm(2000*2000), Dim = as.integer(c(2000,2000)))
  b <- as.double(1:2000)
  invisible(gc())
  timing <- system.time({
    c <- solve(crossprod(a), crossprod(a,b))
  })[3]
  cumulate <- cumulate + timing
  
  # This is the old method
  #a <- Rnorm(600*600); dim(a) <- c(600,600)
  #b <- 1:600
  #invisible(gc())
  #timing <- system.time({
  #  qra <- qr(a, tol = 1e-7);
  #  c <- qr.coef(qra, b)
  #  #Rem: a little faster than c <- lsfit(a, b, inter=F)$coefficients
  #})[3]
  #cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[5, 1] <- timing
cat(c("Linear regr. over a 3000x3000 matrix (c = a \\ b')___ (sec): ", timing, "\n"))
remove("a", "b", "c", "qra")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

times[ , 1] <- sort(times[ , 1])
cat("                      --------------------------------------------\n")
cat(c("                 Trimmed geom. mean (2 extremes eliminated): ", exp(mean(log(times[2:4, 1]))), "\n\n"))

cat("   II. Matrix functions\n")
cat("   --------------------\n")
if (R.Version()$os == "Win32") flush.console()

# (1)
cumulate <- 0; b <- 0
for (i in 1:runs) {
  a <- Rnorm(2400000)
  invisible(gc())
  timing <- system.time({
    b <- fft(a)
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[1, 2] <- timing
cat(c("FFT over 2,400,000 random values____________________ (sec): ", timing, "\n"))
remove("a", "b")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (2)
cumulate <- 0; b <- 0
for (i in 1:runs) {
  a <- array(Rnorm(600*600), dim = c(600, 600))
  # Only needed if using eigen.Matrix(): Matrix.class(a)
  invisible(gc())
  timing <- system.time({
  	b <- eigen(a, symmetric=FALSE, only.values=TRUE)$Value
  	# Rem: on my machine, it is faster than:
  #	 b <- La.eigen(a, symmetric=F, only.values=T, method="dsyevr")$Value
  #	 b <- La.eigen(a, symmetric=F, only.values=T, method="dsyev")$Value
  #  b <- eigen.Matrix(a, vectors = F)$Value
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[2, 2] <- timing
cat(c("Eigenvalues of a 640x640 random matrix______________ (sec): ", timing, "\n"))
remove("a", "b")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (3)
cumulate <- 0; b <- 0
for (i in 1:runs) {
  a <- Rnorm(2500*2500); dim(a) <- c(2500, 2500)
  #Matrix.class(a)
  invisible(gc())
  timing <- system.time({
    #b <- determinant(a, logarithm=F)
    # Rem: the following is slower on my computer!
    # b <- det.default(a)
    b <- det(a)
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[3, 2] <- timing
cat(c("Determinant of a 2500x2500 random matrix____________ (sec): ", timing, "\n"))
remove("a", "b")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (4)
cumulate <- 0; b <- 0
for (i in 1:runs) {
  a <- crossprod(new("dgeMatrix", x = Rnorm(3000*3000),
                       Dim = as.integer(c(3000, 3000))))
  invisible(gc())
  #a <- Rnorm(900*900); dim(a) <- c(900, 900)
  #a <- crossprod(a, a)
  timing <- system.time({
    b <- chol(a)
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[4, 2] <- timing
cat(c("Cholesky decomposition of a 3000x3000 matrix________ (sec): ", timing, "\n"))
remove("a", "b")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (5)
cumulate <- 0; b <- 0
for (i in 1:runs) {
  a <- new("dgeMatrix", x = Rnorm(1600*1600), Dim = as.integer(c(1600, 1600)))
  invisible(gc())
  #a <- Rnorm(400*400); dim(a) <- c(400, 400)
  timing <- system.time({
  #  b <- qr.solve(a)
    # Rem: a little faster than
    b <- solve(a)
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[5, 2] <- timing
cat(c("Inverse of a 1600x1600 random matrix________________ (sec): ", timing, "\n"))
remove("a", "b")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

times[ , 2] <- sort(times[ , 2])
cat("                      --------------------------------------------\n")
cat(c("                Trimmed geom. mean (2 extremes eliminated): ", exp(mean(log(times[2:4, 2]))), "\n\n"))

cat("   III. Programmation\n")
cat("   ------------------\n")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (1)
cumulate <- 0; a <- 0; b <- 0; phi <- 1.6180339887498949
for (i in 1:runs) {
  a <- floor(Runif(3500000)*1000)
  invisible(gc())
  timing <- system.time({
    b <- (phi^a - (-phi)^(-a))/sqrt(5)
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[1, 3] <- timing
cat(c("3,500,000 Fibonacci numbers calculation (vector calc)(sec): ", timing, "\n"))
remove("a", "b", "phi")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (2)
cumulate <- 0; a <- 3000; b <- 0
for (i in 1:runs) {
  invisible(gc())
  timing <- system.time({
    b <- rep(1:a, a); dim(b) <- c(a, a);
    b <- 1 / (t(b) + 0:(a-1))
    # Rem: this is twice as fast as the following code proposed by R programmers
    # a <- 1:a; b <- 1 / outer(a - 1, a, "+")
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[2, 3] <- timing
cat(c("Creation of a 3000x3000 Hilbert matrix (matrix calc) (sec): ", timing, "\n"))
remove("a", "b")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (3)
cumulate <- 0; c <- 0
gcd2 <- function(x, y) {if (sum(y > 1.0E-4) == 0) x else {y[y == 0] <- x[y == 0]; Recall(y, x %% y)}}
for (i in 1:runs) {
  a <- ceiling(Runif(400000)*1000)
  b <- ceiling(Runif(400000)*1000)
  invisible(gc())
  timing <- system.time({	  
    c <- gcd2(a, b)                            # gcd2 is a recursive function
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[3, 3] <- timing
cat(c("Grand common divisors of 400,000 pairs (recursion)__ (sec): ", timing, "\n"))
remove("a", "b", "c", "gcd2")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (4)
cumulate <- 0; b <- 0
for (i in 1:runs) {
  b <- rep(0, 500*500); dim(b) <- c(500, 500)
  invisible(gc())
  timing <- system.time({
  	# Rem: there are faster ways to do this
  	# but here we want to time loops (220*220 'for' loops)! 
    for (j in 1:500) {
      for (k in 1:500) {
        b[k,j] <- abs(j - k) + 1
      }
    }
  })[3]
  cumulate <- cumulate + timing
}
timing <- cumulate/runs
times[4, 3] <- timing
cat(c("Creation of a 500x500 Toeplitz matrix (loops)_______ (sec): ", timing, "\n"))
remove("b", "j", "k")
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

# (5)
cumulate <- 0; p <- 0; vt <- 0; vr <- 0; vrt <- 0; rvt <- 0; RV <- 0; j <- 0; k <- 0;
x2 <- 0; R <- 0; Rxx <- 0; Ryy <- 0; Rxy <- 0; Ryx <- 0; Rvmax <- 0
# Calculate the trace of a matrix (sum of its diagonal elements)
Trace <- function(y) {sum(c(y)[1 + 0:(min(dim(y)) - 1) * (dim(y)[1] + 1)], na.rm=FALSE)}
for (i in 1:runs) {
  x <- abs(Rnorm(45*45)); dim(x) <- c(45, 45)
  invisible(gc())
  timing <- system.time({
    # Calculation of Escoufier's equivalent vectors
    p <- ncol(x)
    vt <- 1:p                                  # Variables to test
    vr <- NULL                                 # Result: ordered variables
    RV <- 1:p                                  # Result: correlations
    vrt <- NULL
    for (j in 1:p) {                           # loop on the variable number
      Rvmax <- 0
      for (k in 1:(p-j+1)) {                   # loop on the variables
        x2 <- cbind(x, x[,vr], x[,vt[k]])
        R <- cor(x2)                           # Correlations table
        Ryy <- R[1:p, 1:p]
        Rxx <- R[(p+1):(p+j), (p+1):(p+j)]
        Rxy <- R[(p+1):(p+j), 1:p]
        Ryx <- t(Rxy)
        rvt <- Trace(Ryx %*% Rxy) / sqrt(Trace(Ryy %*% Ryy) * Trace(Rxx %*% Rxx)) # RV calculation
        if (rvt > Rvmax) {
          Rvmax <- rvt                         # test of RV
          vrt <- vt[k]                         # temporary held variable
        }
      }
      vr[j] <- vrt                             # Result: variable
      RV[j] <- Rvmax                           # Result: correlation
      vt <- vt[vt!=vr[j]]                      # reidentify variables to test
    }
  })[3]
  cumulate <- cumulate + timing
}
times[5, 3] <- timing
cat(c("Escoufier's method on a 45x45 matrix (mixed)________ (sec): ", timing, "\n"))
remove("x", "p", "vt", "vr", "vrt", "rvt", "RV", "j", "k")
remove("x2", "R", "Rxx", "Ryy", "Rxy", "Ryx", "Rvmax", "Trace") 
if (R.Version()$os == "Win32" || R.Version()$os == "mingw32") flush.console()

times[ , 3] <- sort(times[ , 3])
cat("                      --------------------------------------------\n")
cat(c("                Trimmed geom. mean (2 extremes eliminated): ", exp(mean(log(times[2:4, 3]))), "\n\n\n"))

cat(c("Total time for all 15 tests_________________________ (sec): ", sum(times), "\n"))
cat(c("Overall mean (sum of I, II and III trimmed means/3)_ (sec): ", exp(mean(log(times[2:4, ]))), "\n"))
remove("cumulate", "timing", "times", "runs", "i")
cat("                      --- End of test ---\n\n")   



   R Benchmark 2.5
Number of times each test is run__________________________:  3

   I. Matrix calculation
   ---------------------
Creation, transp., deformation of a 2500x2500 matrix (sec):  0.865000000000085 
2400x2400 normal distributed random matrix ^1000____ (sec):  0.636666666666618 
Sorting of 7,000,000 random values__________________ (sec):  1.00933333333342 
2800x2800 cross-product matrix (b = a' * a)_________ (sec):  0.940333333333153 
Linear regr. over a 3000x3000 matrix (c = a \ b')___ (sec):  0.717999999999999 
                      --------------------------------------------
                 Trimmed geom. mean (2 extremes eliminated):  0.835873957160623 

   II. Matrix functions
   --------------------
FFT over 2,400,000 random values____________________ (sec):  0.392666666666779 
Eigenvalues of a 640x640 random matrix______________ (sec):  0.664333333333161 
Determinant of a 2500x2500 random matrix____________ (sec):  0.429999999999836 
Cholesky decomposition of a 3

## Code Replication Source

From A Pedigree-Based Reaction Norm Model for Prediction of Cotton Yield in Multienvironment Trials

https://dl.sciencesocieties.org/publications/cs/abstracts/55/3/1143?access=0&view=pdf

### Description of Data Objects

* `Y`: data frame containing the elements described below;

  * `Y$grain_yield` (n×1), a numeric vector with centered and standardized yield;

  * `Y$VAR`  (n×1), a factor giving the IDs for the varieties;

  * `Y$ENV` (n×1), a factor giving the IDs for the environments (year-location);

* `A`: a symmetric positive semi-definite matrix containing the pedigree or marker-based relationships (dimensions equal to number of lines by number of lines). We assume that the `rownames(A)=colnames(A)` gives the IDs of the lines;

* `W`: a matrix containing the environmental covariates (n×q).


## Processing the Entire Dataset

### Import and Process CIMMYT Data

In [0]:
#data = pd.read_pickle('proc_data.pkl')
# data.columns=data.columns.str.upper()
# data.head()

In [0]:
# Save Data from Python Environment to CSV
# data['GRAIN_YIELD'] = pd.to_numeric(data['GRAIN YIELD'],errors='coerce')
# data.drop_duplicates(subset=['UNIQUE_ID'],inplace=True)
# data.dropna(subset=['GRAIN_YIELD'],inplace=True)
# data.dropna(subset=['HARVEST_FINISHING_DATE'],inplace=True)
# data.dropna(subset=['GENO_ID'],inplace=True)

# data = data.astype(str)
# data.to_csv('yield_data.csv')

In [0]:
%%R
# Import Data Objects to R Environment
# n is number of observations

# data = read.csv('proc_data.csv', header = TRUE)
# str(data)

In [0]:

# data = data.frame(lapply(data, as.character), stringsAsFactors=FALSE)
# data[ data == "nan" ] <- NA
# data[ data == "NaN" ] <- NA
# data[ data == "NaT" ] <- NA
# row.names(data) = data$UNIQUE_ID

# nrow(data)

In [0]:
%%R
# Import GID Numbers CSV
# GID_CID_SID = read.csv('GID_Numbers.csv')

# GID_CID_SID = data.frame(lapply(GID_CID_SID, as.character), stringsAsFactors=FALSE)
# GID_CID_SID[ GID_CID_SID == "nan" ] <- NA
# GID_CID_SID[ GID_CID_SID == "NaN" ] <- NA
# GID_CID_SID[ GID_CID_SID == "NaT" ] <- NA

# GID_CID_SID$GENO_ID = paste(GID_CID_SID$CID, "_", GID_CID_SID$SID, sep = "")

In [0]:
%%R
# Join Sample with GIDs by CID_SID
# data_merge = merge(data, GID_CID_SID, by="GENO_ID", )
# data_merge = data.frame(lapply(data_merge, as.character), stringsAsFactors=FALSE)
# data_merge[ data_merge == "nan" ] <- NA
# data_merge[ data_merge == "NaN" ] <- NA
# data_merge[ data_merge == "NaT" ] <- NA
# row.names(data_merge) = data_merge$UNIQUE_ID

# nrow(data_merge)

In [0]:
%%R
# Drop rows with missing data for yield, location, harvest timing
# data_merge = data_merge[!is.na(data_merge$GRAIN_YIELD),]
# data_merge = data_merge[!is.na(data_merge$LAT_COORD),]
# data_merge = data_merge[!is.na(data_merge$LONG_COORD),]
# data_merge = data_merge[!is.na(data_merge$HARVEST_FINISHING_DATE),]


In [0]:
%%R
data_merge = read.csv('data_merge.csv')
str(data_merge)

In [0]:
%%R
# Identify missing IDs
Un_ID_missing = setdiff(rownames(data), rownames(data_merge))
length(Un_ID_missing)

### Import A Matrix

In [0]:
%%R
# A : pedigree coefficient of parentage symmetric positive semi-definite matrix

# Rdata files don't seem to open properly in the Rpy2 environment
# Use the following commands in R to convert the Rdata to a csv

# A_raw = load('A_3k.RData')
# write.csv(A_raw, 'A_matrix.csv')

A = read.csv('A_matrix.csv', row.names = 1, header = TRUE)

In [0]:
%%R
# Remove the X prefix from the column names
names(A) <- substring(names(A),2)

# Get a list of unique GID values from the data
Unique_GID = c(unique(data_merge$GID))
length(Unique_GID)


In [0]:
%%R
# Identify the GIDs which are present in the data but not the A matrix
GID_missing = setdiff(Unique_GID, rownames(A))
length(GID_missing)

In [0]:
%%R
# Remove the missing GID from the list of unique GIDs from the data sample
if (length(GID_missing) > 0) {
Unique_GID = Unique_GID[Unique_GID != GID_missing]
}  

length(Unique_GID)


In [0]:
%%R
# Subset the A matrix to the list of GIDs in the data sample
A = A[Unique_GID,Unique_GID]

# Convert to a matrix
A = as.matrix(A)


In [0]:
%%R
# Remove the missing GID values from the data sample
data_merge = data_merge[ ! data_merge$GID %in% GID_missing, ]
print(paste('The number of rows in the data sample is: ', nrow(data_merge)))
print(paste('The number of unique varieties by GID is: ', length(unique(data_merge$GID))))

### Splitting into Yield and Environment Objects


In [0]:
%%R
# Yield Vector (nx1)
grain_yield = data_merge$GRAIN_YIELD
grain_yield = as.numeric(grain_yield)
grain_yield = standardize(grain_yield)
length(grain_yield)

In [0]:
%%R
# Varieties (nx1)
VAR = data_merge$GID
VAR = as.numeric(VAR)


In [0]:
%%R
# Environments, meaning location-year combinations (nx1)
data_merge$LOC_YEAR = paste(data_merge$LAT_COORD, "_",
                            data_merge$LONG_COORD, "_", 
                            sapply(strsplit(data_merge$HARVEST_FINISHING_DATE,"-"), `[`, 1)
                            , sep = "")

ENV = data_merge$LOC_YEAR

In [0]:
%%R
Y = data.frame(grain_yield, VAR, ENV)
print(nrow(Y))
head(Y)

In [0]:
%%R
# Extract Environmental Covariates
env_cov_list = c('ALTITUDE',
'PPN_10TH_MO_BEFORE_HARVESTED',
'PPN_11TH_MO_BEFORE_HARVESTED',
'PPN_1ST_MO_BEFORE_HARVESTED',
'PPN_2ND_MO_BEFORE_HARVESTED',
'PPN_3RD_MO_BEFORE_HARVESTED',
'PPN_4TH_MO_BEFORE_HARVESTED',
'PPN_5TH_MO_BEFORE_HARVESTED',
'PPN_6TH_MO_BEFORE_HARVESTED',
'PPN_7TH_MO_BEFORE_HARVESTED',
'PPN_8TH_MO_BEFORE_HARVESTED',
'PPN_9TH_MO_BEFORE_HARVESTED',
'PPN_MONTH_OF_HARVESTED',
'PRECIPITATION_FROM_SOWING_TO_MATURITY',
'TOTAL_PRECIPIT_IN_12_MONTHS',
'IRRIGATED.YES'
)

W = data_merge[env_cov_list]
W = type.convert(W)

# dmy = dummyVars(" ~ .", data = W)
# W = data.frame(predict(dmy, newdata = W))

str(W)

In [0]:
%%R
# Visualize missing data using mice_plot function
mice_plot <- aggr(W, col=c('navyblue','yellow'),
                    numbers=TRUE, sortVars=TRUE,
                    labels=names(W), cex.axis=.7,
                    gap=3, ylab=c("Missing data","Pattern"))

### MOVE TO PROCESSING: Multiple Imputation by Chained Equations for Environment Data

In [0]:
%%R
# Hmisc method, doesn't work unless you spell out all the variables in the function.
# Potentially find a workaround

#install.packages("Hmisc")
#library(Hmisc)

#ivs = paste(env_cov_list,sep = "" )
#iv_string <- paste(ivs, collapse=" + ")
#iv_string[0]

#impute_arg <- aregImpute(as.formula(~iv_string), data = W, n.impute = 5)

In [0]:
%%R
# missForest function for imputation
# Seems to be the fastest and works non-parametrically

# install.packages("missForest")
# library(missForest)


In [0]:
%%R
# W.imp <- missForest(W)

# W.imp$OOBerror


In [0]:
%%R
# W_imp <- W.imp$ximp
# W <- W_imp

In [0]:
%%R
# Save after imputing missing values for W since the function takes a while
# Import the data again if you come back in another session

# write.csv(W, 'W_imp.csv')
#W <- read.csv('W_imp.csv')

In [0]:
%%R
# mice package for imputation, highly recommended but seems too slow for size of data
# https://www.analyticsvidhya.com/blog/2016/03/tutorial-powerful-packages-imputing-missing-values/

#W.imp <- mice(W, m=5, maxit = 15, method = 'pmm')
#summary(W.imp)

In [0]:
%%R
#W <- complete(W.imp,2)

In [0]:
%%R
# Standardize environmental data
W <- standardize(W)
W.head()

In [0]:
%%R
# Check that yield, variety, location, and environmental covariates are same length
print(paste('Yield vector length: ', length(Y$grain_yield)))
print(paste('Varieties vector length: ', length(Y$VAR)))
print(paste('Location-year vector length: ', length(Y$ENV)))
print(paste('Environmental covariates vector length: ', nrow(W)))

# Check that A Matrix contrains all unique varieties from the data
print(paste('Unique Varieties: ', length(unique(Y$VAR))))
print(paste('A Matrix length: ', nrow(A)))

### Export Merged Data for Comparison

In [0]:
%%R
# Export the merged data for comparison
write.csv(data_merge, 'merged_data.csv')
write.csv(Y, 'pheno_yield_data.csv')
write.csv(W, 'environmental_covariates_data.csv')
write.csv(A, 'pedigree_data.csv')

## Import Data and Prep for Analysis

## Import Data from CSV

In [0]:
%%R
# Read data back in, only if you have already saved it in a previous session and have to come back
Y <- read.csv('pheno_yield_data.csv', row.names = 1)
W <- read.csv('environmental_covariates_data.csv', row.names = "X")
A <- read.csv('pedigree_data.csv', row.names = 1)

In [0]:
%%R
Y$ENV <- sapply(Y$ENV, as.factor)

In [114]:
%%R
str(Y)

'data.frame':	145348 obs. of  3 variables:
 $ grain_yield: num  2.065 5.764 3.697 2.993 0.861 ...
 $ VAR        : int  1395073 1395073 1395073 1395073 1395073 1395073 1395073 1395073 1395073 1395073 ...
 $ ENV        : Factor w/ 2326 levels "25.18_83.03_2003",..: 1 2 3 4 5 6 7 8 9 10 ...


In [121]:
%%R
head(Y$ENV)

NotImplementedError: ignored

In [95]:
%%R
str(W)

'data.frame':	145348 obs. of  18 variables:
 $ X.1                                  : int  1 2 3 4 5 6 7 8 9 10 ...
 $ ALTITUDE                             : int  128 985 540 403 1600 477 490 497 110 1198 ...
 $ PPN_10TH_MO_BEFORE_HARVESTED         : num  71.5 13.4 80 37.1 48.3 ...
 $ PPN_11TH_MO_BEFORE_HARVESTED         : num  55.22 6.37 82 20.93 36.16 ...
 $ PPN_1ST_MO_BEFORE_HARVESTED          : num  26.5 23.9 48 9 25.8 ...
 $ PPN_2ND_MO_BEFORE_HARVESTED          : num  91 40 31 53 28.9 ...
 $ PPN_3RD_MO_BEFORE_HARVESTED          : num  10 62 57 109 34.3 ...
 $ PPN_4TH_MO_BEFORE_HARVESTED          : num  4 35 103 43 30.5 ...
 $ PPN_5TH_MO_BEFORE_HARVESTED          : num  9.44 30 31 27 19.54 ...
 $ PPN_6TH_MO_BEFORE_HARVESTED          : num  9.88 30 150 28.73 22.29 ...
 $ PPN_7TH_MO_BEFORE_HARVESTED          : num  35.6 19 286 38.8 47.6 ...
 $ PPN_8TH_MO_BEFORE_HARVESTED          : num  87.7 22 67 48.9 59.7 ...
 $ PPN_9TH_MO_BEFORE_HARVESTED          : num  66.5 1 236 31.8 46.6 ...
 

In [98]:
%%R
str(A[,0:5])

'data.frame':	3489 obs. of  5 variables:
 $ X30374: num  1 0 0 0 0 0 0 0 0 0 ...
 $ X78690: num  0 1 0 0 0 0 0 0 0 0 ...
 $ X15292: num  0 0 1.5 0 0 0 0 0 0 0 ...
 $ X67229: num  0 0 0 1.5 0 0 0 0 0 0 ...
 $ X67418: num  0 0 0 0 1.5 0 0 0 0 0 ...


In [99]:
%%R
# A matrix doesn't read in well as a CSV, need to repeat some processing steps
# Remove the X prefix from the column names
names(A) <- substring(names(A),2)
print(colnames(A)[1])
print(rownames(A)[1])

[1] "30374"
[1] "30374"


In [100]:
%%R
# Get a list of unique GID values from the data
Unique_GID = c(unique(Y$VAR))
length(Unique_GID)


[1] 3443


In [101]:
%%R
# Identify the GIDs which are present in the data but not the A matrix
#GID_missing = setdiff(Unique_GID, rownames(A))
GID_missing = setdiff(rownames(A), Unique_GID)
length(GID_missing)

[1] 46


In [102]:
%%R
# Remove the missing GID from the list of unique GIDs from the data sample
Unique_GID = Unique_GID[Unique_GID != GID_missing]
Unique_GID <- unlist(lapply(Unique_GID, as.character))
# Subset the A matrix to the list of GIDs in the data sample
A = A[Unique_GID,Unique_GID]
# Convert to a matrix
A = as.matrix(A)
nrow(A)

[1] 3443


In [103]:
%%R
print(paste('Environmental covariates vector length: ', nrow(W)))
print(paste('Unique Varieties: ', length(unique(Y$VAR))))
print(paste('A Matrix length: ', nrow(A)))

[1] "Environmental covariates vector length:  145348"
[1] "Unique Varieties:  3443"
[1] "A Matrix length:  3443"


In [0]:
%%R
write.csv(Y, 'pheno_yield_data_adj.csv')
write.csv(W, 'environmental_covariates_data_adj.csv')
write.csv(A, 'pedigree_data_adj.csv')

In [0]:
%%R
remove(Y,W,A)

## Convert Data Matrices to FF Connected Objects

In [0]:
%%capture
%%R
install.packages('ff')
install.packages('ffbase')
install.packages('LaF')

In [0]:
%%R
library(LaF)
library(ffbase)
library(ff)

In [0]:
%%R
Y_laf <- laf_open_csv('pheno_yield_data_adj.csv', 
  column_names = c('index','grain_yield', 'VAR', 'ENV'), 
  column_types = c('integer','numeric','integer','factor'),
  dec=".", sep=",", skip=1)

Y_ffdf <- laf_to_ffdf(Y_laf)

In [118]:
%%R
str(Y_ffdf)

List of 3
 $ virtual: 'data.frame':	4 obs. of  7 variables:
 .. $ VirtualVmode     : chr  "integer" "double" "integer" "integer"
 .. $ AsIs             : logi  FALSE FALSE FALSE FALSE
 .. $ VirtualIsMatrix  : logi  FALSE FALSE FALSE FALSE
 .. $ PhysicalIsMatrix : logi  FALSE FALSE FALSE FALSE
 .. $ PhysicalElementNo: int  1 2 3 4
 .. $ PhysicalFirstCol : int  1 1 1 1
 .. $ PhysicalLastCol  : int  1 1 1 1
 .. - attr(*, "Dim")= int  145348 4
 .. - attr(*, "Dimorder")= int  1 2
 $ physical: List of 4
 .. $ index      : list()
 ..  ..- attr(*, "physical")=Class 'ff_pointer' <externalptr> 
 ..  .. ..- attr(*, "vmode")= chr "integer"
 ..  .. ..- attr(*, "maxlength")= int 145348
 ..  .. ..- attr(*, "pattern")= chr "ffdf"
 ..  .. ..- attr(*, "filename")= chr "/tmp/Rtmp57JxGl/ffdf7d622ff855.ff"
 ..  .. ..- attr(*, "pagesize")= int 65536
 ..  .. ..- attr(*, "finalizer")= chr "delete"
 ..  .. ..- attr(*, "finonexit")= logi TRUE
 ..  .. ..- attr(*, "readonly")= logi FALSE
 ..  .. ..- attr(*, "cach

In [120]:
%%R
head(Y_ffdf$ENV)

NotImplementedError: ignored

In [112]:
%%R
Y_ffdf$ENV <- sapply(Y_ffdf$ENV, as.character)

R[write to console]: Error in `[[<-.ffdf`(`*tmp*`, i, value = list()) : 
  assigned value must be ff
Calls: <Anonymous> ... withVisible -> $<- -> $<-.ffdf -> [[<- -> [[<-.ffdf




Error in `[[<-.ffdf`(`*tmp*`, i, value = list()) : 
  assigned value must be ff
Calls: <Anonymous> ... withVisible -> $<- -> $<-.ffdf -> [[<- -> [[<-.ffdf


In [0]:
%%R
Y_m <- detect_dm_csv("pheno_yield_data_adj.csv")
Y_con <- laf_open(Y_m)
Y_ffdf <- laf_to_ffdf(Y_con)

In [108]:
%%R
head(Y_ffdf)

ffdf (all open) dim=c(145349,4), dimorder=c(1,2) row.names=NULL
ffdf virtual mapping
   PhysicalName VirtualVmode PhysicalVmode  AsIs VirtualIsMatrix
V1           V1      integer       integer FALSE           FALSE
V2           V2      integer       integer FALSE           FALSE
V3           V3      integer       integer FALSE           FALSE
V4           V4      integer       integer FALSE           FALSE
   PhysicalIsMatrix PhysicalElementNo PhysicalFirstCol PhysicalLastCol
V1            FALSE                 1                1               1
V2            FALSE                 2                1               1
V3            FALSE                 3                1               1
V4            FALSE                 4                1               1
   PhysicalIsOpen
V1           TRUE
V2           TRUE
V3           TRUE
V4           TRUE
ffdf data
                       V1                 V2                 V3
1      NA                 grain_yield        VAR               
2      

## Run Model on Full Data

In [0]:
%%R
# Fitting Model 1 (Main Environment-Line Effects)
# (EL)

set.seed(123)
training.samples <- Y$grain_yield %>%
  createDataPartition(p = 0.8, list = FALSE)

y_train = Y$grain_yield
y_train[-training.samples] = NA

# incidence matrix for main eff. of environments.
ZE = model.matrix(~factor(Y$ENV)-1)     

# incidence matrix for main eff. of lines.
Y$VAR = factor(x=Y$VAR,levels=rownames(A),ordered=TRUE)
ZVAR = model.matrix(~Y$VAR-1)

In [0]:
%%capture
%%R
# Model Fitting

ETA = list(ENV=list(X=ZE,model='BRR'),
           VAR=list(X=ZVAR,model='BRR'))

fm1 = BGLR(y=y_train,ETA=ETA,saveAt='M1_',nIter=500,burnIn=200)


In [0]:
%%R
predictions_1 = fm1$yHat[-training.samples]
test_values = Y$grain_yield[-training.samples]
R2_1 = R2(predictions_1, test_values)
RMSE_1 = RMSE(predictions_1, test_values)
MAE_1 = MAE(predictions_1, test_values)
model1_scores = data.frame(R2_1, RMSE_1, MAE_1)
print(model1_scores)

In [0]:
plot(test_values,predictions_1)

In [0]:
!pip install psutil

In [0]:
import psutil

In [0]:
values = psutil.virtual_memory()
avail = values.available >> 30
avail

397

In [0]:
%%R
remove(ETA, fm1)
remove(predictions_1, R2_1, RMSE_1, MAE_1)

In [0]:
%%R
# Alternative Model 2 with Eigendecomposition
L_star = eigen(A, symmetric =TRUE)
Gamma = L_star$vectors
Lambda = L_star$values 
Z_star = ZVAR%*%Gamma%*%(Lambda^0.5)

In [0]:
values = psutil.virtual_memory()
avail = values.available >> 30
avail

393

In [0]:
%%R
remove(A, L_star, Gamma, Lambda)

In [0]:
%%capture
%%R
# Fitting Model 2 with Alternative Z_star (Main Environment-Pedigree Effects)
# (EA)

ETA = list(ENV=list(X=ZE,model='BRR'),
           PED=list(X=Z_star,model='BRR'))

fm2 = BGLR(y=y_train,ETA=ETA,saveAt='M2_',nIter=1000,burnIn=500)

In [0]:
%%R
predictions_2 = fm2$yHat[-training.samples]

R2_2 = R2(predictions_2, test_values)
RMSE_2 = RMSE(predictions_2, test_values)
MAE_2 = MAE(predictions_2, test_values)
model2_scores = data.frame(R2_2, RMSE_2, MAE_2)

In [0]:
%%R
print(model2_scores)
plot(test_values,predictions_2)

In [0]:
%%R
remove(ETA, fm2)
remove(predictions_2, R2_2, RMSE_2,MAE_2)

In [0]:
%%R
# Fitting Model 2 (Main Environment-Pedigree Effects)
# (EA)

# A = A/mean(diag(A))
# L = t(chol(A))
# ZL = ZVAR%*%L

# ETA = list(ENV=list(X=ZE,model='BRR'),
#            PED=list(X=ZL,model='BRR'))
	
# fm2 = BGLR(y=y_train,ETA=ETA,saveAt='M2_',nIter=500,burnIn=200)


In [0]:
%%R
# predictions = fm2$yHat[-training.samples]
# test_values = Y$grain_yield[-training.samples]
# print(data.frame( R2 = R2(predictions, test_values),
#             RMSE = RMSE(predictions, test_values),
#             MAE = MAE(predictions, test_values)))

# plot(test_values,predictions)

In [0]:
%%R
# Fitting Model 3 (Main Environment-Pedigree-Environmental Covariate Effects)
# (EAW)

W = W/sqrt(ncol(W))


In [0]:
values = psutil.virtual_memory()
avail = values.available >> 30
avail

393

In [0]:
%%capture
%%R
ETA = list(ENV=list(X=ZE,model='BRR'),
           PED=list(X=Z_star,model='BRR'),
           EC=list(X=W,model='BRR'))

fm3 = BGLR(y=y_train,ETA=ETA,saveAt='M3_',nIter=500,burnIn=200)

In [0]:
%%R
predictions_3 = fm3$yHat[-training.samples]
test_values = Y$grain_yield[-training.samples]
model3_scores = data.frame(R2_3 = R2(predictions_3, test_values),
            RMSE_3 = RMSE(predictions_3, test_values),
            MAE_3 = MAE(predictions_3, test_values))

print(model3_scores)
plot(test_values,predictions_3)

In [0]:
%%R
remove(ETA, fm3)

In [0]:
values = psutil.virtual_memory()
avail = values.available >> 30
avail

In [0]:
np.intp

In [0]:
%%R
# Fitting Model 4 (Main Environment-Pedigree-Environmental Covariate Effects 
# with Pedigree-Environmental Covariate Interaction)
# (EAW-AxW)

ZAZ = tcrossprod(Z_star)

In [0]:
values = psutil.virtual_memory()
avail = values.available >> 30
avail

239

In [0]:
%%R
W = as.matrix(W)

In [0]:
%%R
# MemoryError: Unable to allocate 157. GiB for an array with shape (145348, 145348) and data type float64
WW = tcrossprod(W)

MemoryError: Unable to allocate 157. GiB for an array with shape (145348, 145348) and data type float64

In [0]:
values = psutil.virtual_memory()
avail = values.available >> 30
avail

82

In [0]:
%%R
remove(W, Z_star)

In [0]:
%%R
K = ZAZ*WW
diag(K) = diag(K)+1/200 
K = K/mean(diag(K))
L2 = t(chol(K))

R[write to console]: Error: cannot allocate vector of size 157.4 Gb

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In Unique_GID != GID_missing :
R[write to console]: 
 
R[write to console]:  longer object length is not a multiple of shorter object length




Error: cannot allocate vector of size 157.4 Gb


In [0]:
%%capture
%%R

ETA = list(ENV=list(X=ZE,model='BRR'),
           PED=list(X=ZL,model='BRR'),
           EC=list(X=W,model='BRR'),
           AxW=list(X=L2,model='BRR'))

fm4 = BGLR(y=y_train,ETA=ETA, saveAt='M4_',nIter=500,burnIn=200)

In [0]:
predictions_4 = fm4$yHat[-training.samples]
print(data.frame( R2_4 = R2(predictions_4, test_values),
            RMSE_4 = RMSE(predictions_4, test_values),
            MAE_4 = MAE(predictions_4, test_values)))

plot(test_values,predictions_4)

In [0]:
%%R
# Fitting Model 5 (Main Environment-Pedigree-Environmental Covariate Effects 
# with Pedigree-Environmental Covariate Interaction and Pedigree-Environment Main Effects)
# (EAW-AxW-AxE)

ZZ = tcrossprod(ZE)
K = ZZ* ZAZ
diag(K) = diag(K)+1/200 
K = K/mean(diag(K))
L3 = t(chol(K))

In [0]:
%%capture
%%R

ETA = list(ENV=list(X=ZE,model='BRR'),
           PED=list(X=ZL,model='BRR'),
           EC=list(X=W,model='BRR'),
           AxW=list(X=L2,model='BRR'),
           AxE=list(X=L3,model='BRR'))

fm5 = BGLR(y=y_train,ETA=ETA, saveAt='M5_',nIter=500,burnIn=200)

In [0]:
%%R
predictions_5 = fm5$yHat[-training.samples]

model5_results =data.frame( R2_5 = R2(predictions_5, test_values),
            RMSE_5 = RMSE(predictions_5, test_values),
            MAE_5 = MAE(predictions_5, test_values))

print(model5_results)

plot(test_values,predictions_5)